# Alt Energy Equity Data - Feature Engineering

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('white')

In [4]:
idx = pd.IndexSlice

## Get Data

In [5]:
START = '2019-01-06'
END = '2021-01-06'

In [6]:
DATA_STORE = '../Data_Processing/Data/H5/assets.h5'
ohlcv = ['Open', 'Close', 'Low', 'High', 'Volume']
with pd.HDFStore(DATA_STORE) as store:
    prices = (store['Alt_Energy/prices']
              .loc[idx[START:END, :], ohlcv]
              .swaplevel()
              .sort_index())
    prices.index.set_names(["Ticker", "date"], inplace=True)

In [7]:
prices.info(null_counts=True)    

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5544 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5544 non-null   float64
 1   Close   5544 non-null   float64
 2   Low     5544 non-null   float64
 3   High    5544 non-null   float64
 4   Volume  5544 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 236.9+ KB


In [8]:
before = len(prices.index.unique('Ticker').unique())

### Remove symbols with missing values

In [9]:
prices = (prices.unstack('Ticker')
        .sort_index()
        .ffill(limit=5)
        .dropna(axis=1)
        .stack('Ticker')
        .swaplevel())
prices.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5544 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5544 non-null   float64
 1   Close   5544 non-null   float64
 2   Low     5544 non-null   float64
 3   High    5544 non-null   float64
 4   Volume  5544 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 236.9+ KB


In [10]:
after = len(prices.index.unique('Ticker').unique())
print(f'Before: {before:,.0f} after: {after:,.0f}')

Before: 11 after: 11


### Keep most traded symbols

In [11]:
dv = prices.Close.mul(prices.Volume)
keep = dv.groupby('Ticker').median().nlargest(1000).index.tolist()

In [12]:
prices = prices.loc[idx[keep, :], :]
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5544 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5544 non-null   float64
 1   Close   5544 non-null   float64
 2   Low     5544 non-null   float64
 3   High    5544 non-null   float64
 4   Volume  5544 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 236.9+ KB


## Feature Engineering

### Compute period returns

In [13]:
intervals = [1, 5, 10, 21, 63]

In [14]:
returns = []
by_ticker = prices.groupby(level='Ticker').Close
for t in intervals:
    returns.append(by_ticker.pct_change(t).to_frame(f'ret_{t}'))
returns = pd.concat(returns, axis=1)

In [15]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5544 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ret_1   5533 non-null   float64
 1   ret_5   5489 non-null   float64
 2   ret_10  5434 non-null   float64
 3   ret_21  5313 non-null   float64
 4   ret_63  4851 non-null   float64
dtypes: float64(5)
memory usage: 236.9+ KB


### Remove outliers

In [16]:
max_ret_by_sym = returns.groupby(level='Ticker').max()

In [17]:
percentiles = [0.001, .005, .01, .025, .05, .1]
percentiles += [1-p for p in percentiles]
max_ret_by_sym.describe(percentiles=sorted(percentiles)[6:])

,ret_1,ret_5,ret_10,ret_21,ret_63
count,11.000000,11.000000,11.000000,11.000000,11.000000
mean,0.234957,0.428627,0.551158,0.668492,1.452919
std,0.140636,0.251580,0.363394,0.385731,1.000989
min,0.110989,0.184363,0.195967,0.263648,0.390370
50%,0.140817,0.354233,0.410338,0.505352,0.918261
90%,0.444730,0.736156,1.059259,1.177610,2.998500
95%,0.450318,0.878471,1.160121,1.275618,3.025765
97.5%,0.453113,0.949628,1.210551,1.324622,3.039398
99%,0.454789,0.992322,1.240810,1.354025,3.047577
99.5%,0.455348,1.006554,1.250896,1.363826,3.050304


In [18]:
quantiles = max_ret_by_sym.quantile(.95)
to_drop = []
for ret, q in quantiles.items():
    to_drop.extend(max_ret_by_sym[max_ret_by_sym[ret]>q].index.tolist()) 

In [19]:
to_drop = pd.Series(to_drop).value_counts()
to_drop = to_drop[to_drop > 1].index.tolist()
len(to_drop)

1

In [20]:
prices = prices.drop(to_drop, level='Ticker')
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5040 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5040 non-null   float64
 1   Close   5040 non-null   float64
 2   Low     5040 non-null   float64
 3   High    5040 non-null   float64
 4   Volume  5040 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 215.8+ KB


### Calculate relative return percentiles

In [21]:
returns = []
by_sym = prices.groupby(level='Ticker').Close
for t in intervals:
    ret = by_sym.pct_change(t)
    rel_perc = (ret.groupby(level='date')
             .apply(lambda x: pd.qcut(x, q=20, labels=False, duplicates='drop')))
    returns.extend([ret.to_frame(f'ret_{t}'), rel_perc.to_frame(f'ret_rel_perc_{t}')])
returns = pd.concat(returns, axis=1)

### Technical Indicators

#### Percentage Price Oscillator

In [22]:
ppo = prices.groupby(level='Ticker').Close.apply(talib.PPO).to_frame('PPO')

#### Normalized Average True Range

In [23]:
natr = prices.groupby(level='Ticker', group_keys=False).apply(lambda x: talib.NATR(x.High, x.Low, x.Close)).to_frame('NATR')

#### Relative Strength Indicator

In [24]:
rsi = prices.groupby(level='Ticker').Close.apply(talib.RSI).to_frame('RSI')

#### Bollinger Bands

In [25]:
def get_bollinger(x):
    u, m, l = talib.BBANDS(x)
    return pd.DataFrame({'u': u, 'm': m, 'l': l})

In [26]:
bbands = prices.groupby(level='Ticker').Close.apply(get_bollinger)

### Combine Features

In [27]:
data = pd.concat([prices, returns, ppo, natr, rsi, bbands], axis=1)

In [28]:
data['bbl'] = data.Close.div(data.l)
data['bbu'] = data.u.div(data.Close)
data = data.drop(['u', 'm', 'l'], axis=1)

In [29]:
data.bbu.corr(data.bbl, method='spearman')

0.05339926831197073

In [30]:
data = data.drop(prices.columns, axis=1)

### Create time period indicators

In [31]:
dates = data.index.get_level_values('date')
data['weekday'] = dates.weekday
data['month'] = dates.month
data['year'] = dates.year

## Compute forward returns

In [32]:
outcomes = []
by_ticker = data.groupby('Ticker')
for t in intervals:
    k = f'fwd_ret_{t:02}'
    outcomes.append(k)
    data[k] = by_ticker[f'ret_{t}'].shift(-t)

In [33]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5040 entries, ('BE', Timestamp('2019-01-07 00:00:00')) to ('VWSYF', Timestamp('2021-01-05 00:00:00'))
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ret_1            5030 non-null   float64
 1   ret_rel_perc_1   5030 non-null   float64
 2   ret_5            4990 non-null   float64
 3   ret_rel_perc_5   4990 non-null   float64
 4   ret_10           4940 non-null   float64
 5   ret_rel_perc_10  4940 non-null   float64
 6   ret_21           4830 non-null   float64
 7   ret_rel_perc_21  4830 non-null   float64
 8   ret_63           4410 non-null   float64
 9   ret_rel_perc_63  4410 non-null   float64
 10  PPO              4790 non-null   float64
 11  NATR             4900 non-null   float64
 12  RSI              4900 non-null   float64
 13  bbl              5000 non-null   float64
 14  bbu              5000 non-null   float64
 15  weekday          5040 non-nu

In [34]:
data.to_hdf('data.h5', 'us/alt_energy_forest/equities')